In [21]:
import arcpy
import pandas as pd
import numpy as np

In [2]:
arcpy.env.workspace = 'geo/lekarny.gdb'
arcpy.env.overwriteOutput = True

In [48]:
#99, 07, 15
buff = {}

cursor = arcpy.da.SearchCursor('lek_99_07_15_pnt', ['SHAPE@X', 'SHAPE@Y', 'addr', 'NAZEV', 'ZNACKA', 'rok'])
for row in cursor:
    addr = row[2]
    nazev = row[3]
    znacka = row[4]
    rok = str(row[5])
    x = row[0]
    y = row[1]
    
    if(addr not in buff):
        buff[addr] = {'x': x, 'y': y,'nazev_1999': np.nan, 'nazev_2007': np.nan, 'nazev_2015': np.nan, 'znacka_1999': np.nan, 'znacka_2007': np.nan, 'znacka_2015': np.nan, 'zmena': np.nan}
    buff[addr]['nazev_' + rok] = nazev
    buff[addr]['znacka_' + rok] = znacka
    

In [49]:
lekarny = pd.DataFrame.from_dict(buff, orient='index')
#drop krachlych
lekarny.dropna(subset=['nazev_2015'], inplace=True)

lekarny.reset_index().rename(columns={'index': 'adresa'}, inplace=True)

In [74]:
def zmena(r99, r07, r15):
    if ((r15 != r07) & (r07 == r07)):
        return 'y'
    elif((r15 != r99) & (r99 == r99)):
        return 'y07'
    else:
        return 'n'

In [72]:
lekarny['zmena'] = lekarny.apply(lambda row: zmena(row['znacka_1999'], row['znacka_2007'], row['znacka_2015']), axis=1)

In [85]:
lekarny.adresa = lekarny.adresa.apply(lambda x: x.lstrip(', '))

In [86]:
lekarny.to_csv('upravena-data/mapa_zmeny.csv', encoding='utf-8', index=False)